# Get data into dictionary

In [1]:
from sklearn import metrics

In [2]:
import scipy.io
import os
import numpy as np
path = 'groundTruthmat/'
dataset = []
files = os.listdir(path)
data_dict = {}
# devide four folds
# s1: P03 – P15
# s2: P16 – P28
# s3: P29 – P41
# s4: P42 – P54
for file in files:
    read = scipy.io.loadmat(path + file)
    data_dict[file] = read['labseqid']

In [3]:
def one_hot():
# create frame_label for one hot encoding
    frame_label = {}
    for i in range(48):
        frame_label[i] = 48*[0]
        frame_label[i][i] = 1
    return frame_label

frame_label = one_hot()

In [4]:
def get_input_x_y(data_dict, trainPortion):
    new_dict = {}
    train_y = []
    for filename, frames in data_dict.items():
        frameLen = len(frames)
        
        inputLen = round(frameLen*trainPortion)  
        inputFrames = frames[:inputLen]
        y = frames[inputLen-1]
        for frame in frames[inputLen:]:
#             print(frame, y)
            if frame != y[0]:
                y[0] = frame[0]
                break
        train_y.append(y)
        new_dict[filename] = inputFrames
    return new_dict, train_y

In [30]:
def add_padding(data_dict):
    maxLen = 0
    count = 0
    for frames in data_dict.values():
        
        if len(frames) > maxLen:
            maxLen = len(frames)
    for filename, frames in data_dict.items():
        data_dict[filename] = (maxLen - len(frames)) * [0] + [i[0] for i in frames]
    return data_dict, maxLen
            

In [31]:
def feature_encoding(input_dict):
    for filename, frames in input_dict.items():
        # get corresponding one-hot encode
        new = []
        for each in frames:
            new.append(frame_label[each])
        input_dict[filename] = new
    return input_dict

In [32]:
def label_encoding(train_y):
    encoded_y = []
    for label in train_y:
        new_label = frame_label[label[0]]
        encoded_y.append(new_label)
    return encoded_y

# LSTM

In [33]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
def lstm_model(frame_len):
    clear_session() 
    model = Sequential()
    model.add(LSTM(100, input_shape=(frame_len,48), return_sequences=False))
    model.add(Dense(48, activation='tanh'))
    model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

# 4-fold cross validation

In [41]:
def train_model(trainX, trainY, model):
    model.fit(np.array(trainX), np.array(trainY), epochs = 10, batch_size = 256)
    

In [35]:
def evaluation(testX, testY, model):
    predictions = model.predict(testX)
    results = []
    count = 0
    for i in range(len(predictions)):
        result = np.array([0] * 48)
        index = predictions[i].argmax(axis=-1)
        result[index] = 1
        pre = metrics.accuracy_score(result, testY[i])
        if pre == 1:
            count += 1
    print(count/len(testY))
    return (count/len(testY))

In [36]:
import copy
def cross_validation(input_dict, encoded_y, model):
    file_count = 0
    s1_x, s2_x, s3_x, s4_x = [], [], [], []
    s1_y, s2_y, s3_y, s4_y = [], [], [], []
    # s1: P03 – P15
    # s2: P16 – P28
    # s3: P29 – P41
    # s4: P42 – P54
    count = 0
    for filename, frames in input_dict.items():
        if int(filename[1:3]) <= 15:
            s1_x.append(input_dict[filename])
            s1_y.append(encoded_y[file_count])
        elif 16 <= int(filename[1:3]) <= 28:
            s2_x.append(input_dict[filename])
            s2_y.append(encoded_y[file_count])
        elif 29 <= int(filename[1:3]) <= 41:
            s3_x.append(input_dict[filename])
            s3_y.append(encoded_y[file_count])
        elif 42 <= int(filename[1:3]) <= 54:
            s4_x.append(input_dict[filename])
            s4_y.append(encoded_y[file_count])
        file_count += 1
        
    splits_x = [s1_x, s2_x, s3_x, s4_x]
    splits_y = [s1_y, s2_y, s3_y, s4_y]
    final_acc = 0
    for i in range(4):
        trainX = None
        trainY = None
        for x in range(4):
            if splits_x[x] != splits_x[i]:
                if trainX == None:
                    trainX = copy.deepcopy(splits_x[x])
                else:
                    trainX += copy.deepcopy(splits_x[x])

        for y in range(4):
            if splits_y[y] != splits_y[i]:
                if trainY == None:
                    trainY = copy.deepcopy(splits_y[y])
                else:
                    trainY += copy.deepcopy(splits_y[y])
        testX = splits_x[i]
        testY = splits_y[i]
        print(np.array(trainX).shape, np.array(trainY).shape, np.array(testX).shape,np.array(testY).shape)
        train_model(trainX, trainY, model)
        final_acc += evaluation(testX, testY, model)
        
    print(final_acc/4) 
    return (final_acc/4) 

In [42]:
from keras.backend import clear_session
input_proportion = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# input_proportion = [0.1, 0.2]
results = []
for proportion in input_proportion:
    input_dict, train_y = get_input_x_y(data_dict, proportion)
    input_dict, frame_len = add_padding(input_dict)
    input_dict = feature_encoding(input_dict)
    encoded_y = label_encoding(train_y)
    model = lstm_model(frame_len)
    result = cross_validation(input_dict, encoded_y, model)
    results.append(result)
    clear_session() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               59600     
_________________________________________________________________
dense (Dense)                (None, 48)                4848      
Total params: 64,448
Trainable params: 64,448
Non-trainable params: 0
_________________________________________________________________
(1460, 974, 48) (1460, 48) (252, 974, 48) (252, 48)
Epoch 1/10
6/6 [==============================] - 7s 1s/step - loss: 0.0684 - accuracy: 0.0027
Epoch 2/10
6/6 [==============================] - 7s 1s/step - loss: 0.0493 - accuracy: 0.0295
Epoch 3/10
6/6 [==============================] - 7s 1s/step - loss: 0.0391 - accuracy: 0.1247
Epoch 4/10
6/6 [==============================] - 7s 1s/step - loss: 0.0332 - accuracy: 0.3555
Epoch 5/10
6/6 [==============================] - 7s 1s/step - loss: 0.0294 - accu

5/5 [==============================] - 12s 2s/step - loss: 0.0141 - accuracy: 0.9867
Epoch 9/10
5/5 [==============================] - 11s 2s/step - loss: 0.0136 - accuracy: 0.9867
Epoch 10/10
5/5 [==============================] - 11s 2s/step - loss: 0.0133 - accuracy: 0.9883
0.9930715935334873
(1136, 1948, 48) (1136, 48) (576, 1948, 48) (576, 48)
Epoch 1/10
5/5 [==============================] - 10s 2s/step - loss: 0.0129 - accuracy: 0.9921
Epoch 2/10
5/5 [==============================] - 10s 2s/step - loss: 0.0126 - accuracy: 0.9921
Epoch 3/10
5/5 [==============================] - 10s 2s/step - loss: 0.0123 - accuracy: 0.9956
Epoch 4/10
5/5 [==============================] - 10s 2s/step - loss: 0.0119 - accuracy: 0.9956
Epoch 5/10
5/5 [==============================] - 10s 2s/step - loss: 0.0115 - accuracy: 0.9956
Epoch 6/10
5/5 [==============================] - 10s 2s/step - loss: 0.0112 - accuracy: 0.9956
Epoch 7/10
5/5 [==============================] - 10s 2s/step - loss: 0.0

5/5 [==============================] - 22s 4s/step - loss: 0.0219 - accuracy: 0.7042
Epoch 5/10
5/5 [==============================] - 22s 4s/step - loss: 0.0216 - accuracy: 0.6907
Epoch 6/10
5/5 [==============================] - 22s 4s/step - loss: 0.0214 - accuracy: 0.7050
Epoch 7/10
5/5 [==============================] - 22s 4s/step - loss: 0.0212 - accuracy: 0.7121
Epoch 8/10
5/5 [==============================] - 22s 4s/step - loss: 0.0209 - accuracy: 0.7177
Epoch 9/10
5/5 [==============================] - 21s 4s/step - loss: 0.0207 - accuracy: 0.7351
Epoch 10/10
5/5 [==============================] - 21s 4s/step - loss: 0.0205 - accuracy: 0.7335
0.6696230598669624
(1279, 3896, 48) (1279, 48) (433, 3896, 48) (433, 48)
Epoch 1/10
5/5 [==============================] - 22s 4s/step - loss: 0.0205 - accuracy: 0.7037
Epoch 2/10
5/5 [==============================] - 22s 4s/step - loss: 0.0203 - accuracy: 0.6998
Epoch 3/10
5/5 [==============================] - 22s 4s/step - loss: 0.0

6/6 [==============================] - 41s 7s/step - loss: 0.0631 - accuracy: 0.0541
Epoch 2/10
6/6 [==============================] - 41s 7s/step - loss: 0.0457 - accuracy: 0.1514
Epoch 3/10
6/6 [==============================] - 41s 7s/step - loss: 0.0374 - accuracy: 0.3541
Epoch 4/10
6/6 [==============================] - 41s 7s/step - loss: 0.0326 - accuracy: 0.4918
Epoch 5/10
6/6 [==============================] - 41s 7s/step - loss: 0.0294 - accuracy: 0.5932
Epoch 6/10
6/6 [==============================] - 41s 7s/step - loss: 0.0271 - accuracy: 0.6445
Epoch 7/10
6/6 [==============================] - 41s 7s/step - loss: 0.0255 - accuracy: 0.6795
Epoch 8/10
6/6 [==============================] - 41s 7s/step - loss: 0.0243 - accuracy: 0.6849
Epoch 9/10
6/6 [==============================] - 40s 7s/step - loss: 0.0234 - accuracy: 0.6856
Epoch 10/10
6/6 [==============================] - 39s 7s/step - loss: 0.0228 - accuracy: 0.7027
0.7301587301587301
(1261, 5845, 48) (1261, 48) (45

5/5 [==============================] - 36s 7s/step - loss: 0.0161 - accuracy: 0.7738
Epoch 5/10
5/5 [==============================] - 36s 7s/step - loss: 0.0161 - accuracy: 0.7905
Epoch 6/10
5/5 [==============================] - 36s 7s/step - loss: 0.0158 - accuracy: 0.8055
Epoch 7/10
5/5 [==============================] - 36s 7s/step - loss: 0.0157 - accuracy: 0.8072
Epoch 8/10
5/5 [==============================] - 35s 7s/step - loss: 0.0155 - accuracy: 0.8072
Epoch 9/10
5/5 [==============================] - 34s 7s/step - loss: 0.0154 - accuracy: 0.8099
Epoch 10/10
5/5 [==============================] - 34s 7s/step - loss: 0.0152 - accuracy: 0.8371
0.7725694444444444
0.7313101374006132
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               59600     
_________________________________________________________________
dense (Dense)   

(1279, 8767, 48) (1279, 48) (433, 8767, 48) (433, 48)
Epoch 1/10
5/5 [==============================] - 70s 14s/step - loss: 0.0129 - accuracy: 0.8741
Epoch 2/10
5/5 [==============================] - 72s 14s/step - loss: 0.0126 - accuracy: 0.8812
Epoch 3/10
5/5 [==============================] - 76s 15s/step - loss: 0.0128 - accuracy: 0.8835
Epoch 4/10
5/5 [==============================] - 69s 14s/step - loss: 0.0127 - accuracy: 0.8835
Epoch 5/10
5/5 [==============================] - 69s 14s/step - loss: 0.0124 - accuracy: 0.8647
Epoch 6/10
5/5 [==============================] - 72s 14s/step - loss: 0.0122 - accuracy: 0.8718
Epoch 7/10
5/5 [==============================] - 72s 14s/step - loss: 0.0120 - accuracy: 0.8679
Epoch 8/10
5/5 [==============================] - 61s 12s/step - loss: 0.0119 - accuracy: 0.8569
Epoch 9/10
5/5 [==============================] - 49s 10s/step - loss: 0.0119 - accuracy: 0.8569
Epoch 10/10
5/5 [==============================] - 49s 10s/step - loss: 0

In [43]:
results

[0.9346375492427392,
 0.9420017031118413,
 0.6593585180819996,
 0.7241083223368788,
 0.7683955406846872,
 0.7602085603136483,
 0.7313101374006132,
 0.7721406072919286,
 0.8289806590800173]

In [ ]:
import tf_models, datase                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
ts, utils, metrics
from utils import imshow_
model, param_str = tf_models.temporal_convs_linear(n_nodes[0], conv, n_classes, n_feat, 
                                                    max_len, causal=causal, return_param_str=True)